In [1]:
from pathlib import Path
import os
import numpy as np
import torch
import torchvision.transforms.functional as T
from matplotlib import pyplot as plt
from torchvision.utils import draw_segmentation_masks, make_grid
import pytorch_lightning as pl
import mlflow
from tqdm import tqdm
import torchmetrics
import torch.nn.functional as F
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks import RichProgressBar, ModelCheckpoint

In [2]:
from inz.data.event import Event, Hold, Test, Tier1, Tier3
from inz.data.data_module import XBDDataModule
from inz.models.unet_basic import UNet
from inz.models.unet_basic_pl import SemanticSegmentor, OrdinalCrossEntropyLoss, DiceLoss

In [3]:
RANDOM_SEED = 123
pl.seed_everything(RANDOM_SEED)
device = torch.device("cuda")
torch.set_float32_matmul_precision("high")

Seed set to 123


In [4]:
dm = XBDDataModule(
    path=Path("data/xBD_processed"),
    events={
        # Tier1: [
        #     Event.hurricane_florence,
        #     Event.hurricane_harvey,
        #     Event.hurricane_matthew,
        #     Event.hurricane_michael,
        # ],
        Tier3: [
            Event.joplin_tornado,
            # Event.moore_tornado,
            # Event.tuscaloosa_tornado
        ],
        # Hold: [
        #     Event.hurricane_florence,
        #     Event.hurricane_harvey,
        #     Event.hurricane_matthew,
        #     Event.hurricane_michael,
        # ],
        # Test: [
        #     Event.hurricane_florence,
        #     Event.hurricane_harvey,
        #     Event.hurricane_matthew,
        #     Event.hurricane_michael,
        # ],
    },
    val_faction=0.15,
    test_fraction=0.0,
    train_batch_size=32,
    val_batch_size=32,
    test_batch_size=32,
)
dm.prepare_data()
dm.setup("fit")

print(
    f"{len(dm.train_dataloader())} train batches, {len(dm.val_dataloader())} val batches, {len(dm.test_dataloader())} test batches, "
)

64 train batches, 12 val batches, 0 test batches, 


In [5]:
aaa_loc = []
aaa_cls = []
for batch in tqdm(dm.train_dataloader()):
    pre_images, pre_masks, post_images, post_masks = batch
    counts_post = torch.bincount(post_masks.argmax(dim=1).reshape(-1), minlength=6)
    aaa_cls.append(counts_post)
    counts_pre = torch.bincount(pre_masks.argmax(dim=1).reshape(-1), minlength=6)
    aaa_loc.append(torch.tensor([counts_pre[0], counts_pre[1:].sum()]))

loc_counts = torch.stack(aaa_loc).sum(dim=0).to(torch.float)
cls_counts = torch.stack(aaa_cls).sum(dim=0).to(torch.float)

print(cls_counts)

loc_weights = loc_counts.sum() / loc_counts
loc_weights = (loc_weights / loc_weights.sum()).cuda()
cls_weights = cls_counts.sum() / cls_counts
cls_weights = (cls_weights / cls_weights.sum()).cuda() * torch.tensor(
    [1, 1, 1, 1, 1, 0]
).cuda()  # last class is "unclassified"

print(f"Localization weights: {loc_weights}\nClassification weights: {cls_weights}")

  8%|▊         | 5/64 [00:02<00:25,  2.35it/s]

In [ ]:
model = SemanticSegmentor(
    model=UNet(in_channels=3, out_channels=6),
    localization_loss=torch.nn.BCEWithLogitsLoss(pos_weight=loc_weights[1]),
    # classification_loss=DiceLoss(weight=cls_weights.cuda()),
    # classification_loss=torch.nn.CrossEntropyLoss(weight=cls_weights.cuda(), reduction='sum'),
    classification_loss=OrdinalCrossEntropyLoss(n_classes=6, weights=cls_weights),
    n_classes=6,
)

/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.
/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'localization_loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['localization_loss'])`.
/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:199: Attribute 'classification_loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['classification_loss'])`.


In [ ]:
mlflow.pytorch.autolog()


f1_checkpoint_callback = ModelCheckpoint(
    save_top_k=1, verbose=True, monitor="f1", mode="max", filename="f1_{epoch}{f1:.5f}{iou:.5f}{loss:.5f}"
)

iou_checkpoint_callback = ModelCheckpoint(
    save_top_k=1, verbose=True, monitor="iou", mode="max", filename="iou_{epoch}{f1:.5f}{iou:.5f}{loss:.5f}"
)

loss_checkpoint_callback = ModelCheckpoint(
    save_top_k=1, verbose=True, monitor="loss", mode="min", filename="loss_{epoch}{f1:.5f}{iou:.5f}{loss:.5f}"
)

trainer = pl.Trainer(
    max_epochs=300,
    callbacks=[
        RichProgressBar(),
        f1_checkpoint_callback,
        iou_checkpoint_callback,
        # loss_checkpoint_callback
    ],
    logger=MLFlowLogger(),
    precision="16-mixed",
)
trainer.fit(model, datamodule=dm)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024/06/10 00:44:00 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1fb9843c165b4facbe195e88eb351fe3', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current pytorch workflow
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃    ┃ Name                ┃ Type                    ┃ Params ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0  │ model               │ UNet                    │  124 M │
│ 1  │ localization_loss   │ BCEWithLogitsLoss       │      0 │
│ 2  │ classification_loss │ OrdinalCrossEntropyLoss │      0 │
│ 3  │ accuracy_loc        │ BinaryAccuracy          │      0 │
│ 4  │ iou_loc             │ MeanIoU                 │      0 │
│ 5  │ f1                  │ MulticlassF1Score       │      0 │
│ 6  │ precision           │ MulticlassPrecision     │      0 │
│ 7  │ recall              │ MulticlassRecall        │      0 │
│ 8  │ iou                 │ MeanIoU                 │      0 │
│ 9  │ f1_per_class        │ MulticlassF1Score       │      0 │
│ 10 │ precision_per_class │ MulticlassPrecision     │      0 │
│ 11 │ recall_per_class    │ MulticlassRecall        │      0 │
│ 12 │ iou_per_class       │ MeanIoU                 │      0 │
└────┴─────────────────────┴─────────────────────────┴────────┘

Trainable params: 124 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 124 M                                                                                                
Total estimated model params size (MB): 496

Output()

2024/06/10 00:45:09 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 
'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored 
metric value is not available.

Epoch 0, global step 64: 'f1' reached 0.24114 (best 0.24114), saving model to './mlruns/126730738902562300/dea35f9836764ec8bb457c1d5c7453a0/checkpoints/f1_epoch=0f1=0.24114iou=0.12071loss=2.31874.ckpt' as top 1
Epoch 0, global step 64: 'iou' reached 0.12071 (best 0.12071), saving model to './mlruns/126730738902562300/dea35f9836764ec8bb457c1d5c7453a0/checkpoints/iou_epoch=0f1=0.24114iou=0.12071loss=2.31874.ckpt' as top 1
Epoch 0, global step 64: 'loss' reached 2.31874 (best 2.31874), saving model to './mlruns/126730738902562300/dea35f9836764ec8bb457c1d5c7453a0/checkpoints/loss_epoch=0f1=0.24114iou=0.12071loss=2.31874.ckpt' as top 1


/home/tomek/inz/inz/.venv/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected 
KeyboardInterrupt, attempting graceful shutdown...